In [1]:
import pandas as pd
import os

df_covs = pd.read_csv('processed_data/paul_covariates.csv')
df_covs.rename({'GEOID':'Fips'}, axis=1, inplace=True)
# df_covs

df_deaths = pd.read_csv('processed_data/naive_county_deaths.csv')
# df_deaths.drop(['Date','County','Pop'], axis=1, inplace=True)
# df_deaths

df = df_deaths.merge(df_covs, on='Fips')

In [ ]:
import numpy as np

for i in range(100):
    np.random.seed(666)

    # # # # # # # # # #
    test_ratio = 0.2
    # # # # # # # # # #

    ###########################
    # # FIPS SPLIT # # # # # #
    ###########################

    fips = df.Fips.unique().tolist()

    test_fips = np.random.choice(fips, int(test_ratio*len(fips)), replace=False)
    train_fips = [x for x in fips if x not in test_fips]
    print(len(test_fips), len(train_fips))

    df_train = df[df.Fips.isin(train_fips)]
    df_test = df[df.Fips.isin(test_fips)]
    print(len(df_test), len(df_train))
    

In [14]:
import numpy as np
np.random.seed(666)

# # # # # # # # # #
test_ratio = 0.2
# # # # # # # # # #

###########################
# # FIPS SPLIT # # # # # #
###########################

fips = df.Fips.unique().tolist()

test_fips = np.random.choice(fips, int(test_ratio*len(fips)), replace=False)
train_fips = [x for x in fips if x not in test_fips]
print(len(test_fips), len(train_fips))

df_train = df[df.Fips.isin(train_fips)]
df_test = df[df.Fips.isin(test_fips)]
print(len(df_test), len(df_train))

###########################
# # DATE SPLIT # # # # # #
###########################

# dates = df.Date.unique().tolist()

# test_dates = np.random.choice(dates, int(test_ratio*len(dates)), replace=False)
# train_dates = [x for x in dates if x not in test_dates]
# # len(test_fips), len(train_fips)

# df_train = df[df.Date.isin(train_dates)]
# df_test = df[df.Date.isin(test_dates)]

20 80
780 3120


In [15]:
df_test

,Date,County,Fips,True_county_deaths,Pop,Pop_ratio,Pred_state_deaths,Naive_county_deaths,POP_x,POP_M,...,GQ_F65p_INST,GQ_F65p_INST_COR,GQ_F65p_INST_JUV,GQ_F65p_INST_NUR,GQ_F65p_INST_OTH,GQ_F65p_NONI,GQ_F65p_NONI_UNI,GQ_F65p_NONI_MIL,GQ_F65p_NONI_OTH,Population
39,2021-02-06,Alexander,37003,59,37497,0.003575,9846,35.201421,36491,18568,...,66,0,0,66,0,9,0,0,9,36444
40,2021-02-13,Alexander,37003,71,37497,0.003575,10485,37.485974,36491,18568,...,66,0,0,66,0,9,0,0,9,36444
41,2021-02-20,Alexander,37003,72,37497,0.003575,11231,40.153073,36491,18568,...,66,0,0,66,0,9,0,0,9,36444
42,2021-02-27,Alexander,37003,74,37497,0.003575,11285,40.346134,36491,18568,...,66,0,0,66,0,9,0,0,9,36444
43,2021-03-06,Alexander,37003,76,37497,0.003575,11494,41.093351,36491,18568,...,66,0,0,66,0,9,0,0,9,36444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3583,2021-11-27,Wake,37183,915,1111761,0.106002,18733,1985.741038,1112883,544184,...,1492,5,0,1394,93,715,0,0,715,1129410
3584,2021-12-04,Wake,37183,910,1111761,0.106002,18813,1994.221222,1112883,544184,...,1492,5,0,1394,93,715,0,0,715,1129410
3585,2021-12-11,Wake,37183,911,1111761,0.106002,18980,2011.923606,1112883,544184,...,1492,5,0,1394,93,715,0,0,715,1129410
3586,2021-12-18,Wake,37183,913,1111761,0.106002,19116,2026.339918,1112883,544184,...,1492,5,0,1394,93,715,0,0,715,1129410


In [16]:
# fips

In [17]:
target = 'True_county_deaths'

covs_to_drop = [target,
                'Fips',
                'County',
                'Date',
#                 'Pop'
#                 'Naive_county_deaths',
#                 'Pred_state_deaths'
               ]

X_train = df_train.drop(covs_to_drop,axis=1)
y_train = df_train[target].values

X_test = df_test.drop(covs_to_drop,axis=1)
y_test = df_test[target].values

y_naive = df_test['Naive_county_deaths'].values

print(len(X_train), len(X_test))


from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state=666)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test) #[:,None]
y_prob = clf.predict_proba(X_test)[:,0]

# y_naive = X_test['Naive_county_deaths'].values

# print(y_test[:10])
# print(y_pred[:10])
# print(y_naive[:10])


from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

for which,pred in [ ['Naive', y_naive], ['RF',y_pred] ]:
    mae = round(mean_absolute_error(y_test, pred), 2)
    rmse = round(mean_squared_error(y_test, pred, squared=False), 2)
    r2 = round(r2_score(y_test, pred), 4)
    print(f"\n* {which} *")
    print(f"\tRMSE: {rmse} \tr2: {r2} \tMAE: {mae}")

3120 780

* Naive *
	RMSE: 229.64 	r2: 0.3665 	MAE: 113.22

* RF *
	RMSE: 316.16 	r2: -0.2007 	MAE: 172.44
